In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np

from dataset import TrainDataset
from model import initialize_model

In [2]:
CHECKPOINT_FOLDER = 'checkpoints/'
DATA_FILE = 'E:/xplore_data/data/images.h5'
DHSGPS_FILE = 'data/dhs_gps.csv'

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
# Size of image that will get fed into neural net
INPUT_SIZE = 333

In [5]:
dtrain = TrainDataset(h5_file=DATA_FILE, dhsgps_file=DHSGPS_FILE, K=INPUT_SIZE)

In [6]:
def train_model(model, dataloader, optimizer, scheduler, num_epochs=4):
    n = len(dataloader.dataset)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        criterion = nn.CrossEntropyLoss(reduction='none')

        # Iterate over data.
        for x, y, stats in dataloader:
            x = x.to(device)
            y = y.to(device)
            stats = stats.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(x)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, y) * stats[:, 3:14].mean(axis=1)
                loss = loss.mean()
                loss.backward()
                optimizer.step()
                scheduler.step()

            # statistics
            running_loss += loss.item() * x.size(0)
            running_corrects += torch.sum(preds == y.data)

        epoch_loss = running_loss / n
        epoch_acc = running_corrects.double() / n

        print('Loss: {:.4f} Acc: {:.4f}'.format(
            epoch_loss, epoch_acc))
    return model


In [7]:
net = initialize_model()
net.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(9, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [8]:
BATCH_SIZE = 4
EPOCHS = 1

# Data loader
dloader = torch.utils.data.DataLoader(dtrain, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
# Create training optimizer
optimizer = optim.Adam(net.parameters(), lr=0.01)
# Optimizer LR decay
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)


In [9]:
net = train_model(net, dloader, optimizer, scheduler, EPOCHS)

Epoch 0/0
----------
Loss: 38.9971 Acc: 0.4871


In [10]:
# checkpoint_name = 'mobilenet_1'
# torch.save(net.state_dict(), os.path.join(CHECKPOINT_FOLDER, checkpoint_name))